# Ensemble learning with top-performing kernels

Code inspiration taken from https://www.kaggle.com/anyexiezouqu/magic-ensemble-top-kernels

## Model-weight combinations of each version

### Original weights
V1: weights = 0.5,0.3,0.2,0.1

V2: weights = 0.5,0.3,0.1,0.1

V3: weights = 0.25,0.25,0.25,0.25

V4: weights = 0.5,0.25,0.25

V6: weights = 0.6,0.2,0.2

V7: weights = 0.5,0.3,0.2

### Our own versions

V1: add base-three-shades as fourth method. Weights = 0.35, 0.15, 0.1, 0.45 (wrong: doesn't sum to 1) - 0.54357

V2: Weights = 0.3, 0.15, 0.1, 0.45 - forecasting_acc,tweedie, poisson, three_shades - 0.47391

V3: weights = 0.25, 0.15, 0.1, 0.6 - forecasting_acc,tweedie, poisson, three_shades (wrong: doesn't sum to 1) - 0.72958

V4: weights = 0.25, 0.15, 0.1, 0.5 - forecasting_acc,tweedie, poisson, three_shades - 0.47351

V5: weights = 0.2, 0.15, 0.1, 0.55 - forecasting_acc,tweedie, poisson, three_shades - 0.47314

V6: weights = 0.15, 0.15, 0.1, 0.6 - forecasting_acc,tweedie, poisson, three_shades - 0.47282

V7: weights = 0.1, 0.2, 0.1, 0.6 - forecasting_acc,tweedie, poisson, three_shades - 0.47235

V8: weights = 0.3, 0.1, 0.6 - tweedie, poisson, three_shades - 0.47199

V9: weights = 0.4, 0.6 - tweedie, three_shades - 0.47210

V10: weights = 0.25, 0.2, 0.55 - tweedie, poisson, three_shades - 0.47235

V11: weights = 0.2, 0.1, 0.5, 0.2 - tweedie, poisson, three_shades, witch - 0.46694

V12: weights = 0.2, 0.1, 0.3, 0.4 - tweedie, poisson, three_shades, witch - 0.46442

V13: weights = 0.2, 0.1, 0.2, 0.5 - tweedie, poisson, three_shades, witch - 0.46344

V14: weights = 0.1, 0.1, 0.2, 0.6 - tweedie, poisson, three_shades, witch - 0.46294

V15: weights = 0.1, 0.1, 0.1, 0.7 - tweedie, poisson, three_shades, witch - 0.46216

V16: weights = 0.1, 0.1, 0.1, 0.7 - tweedie, poisson, groupkfold, witch - 0.46181

V17: weights = 0.1, 0.05, 0.1, 0.75 - tweedie, poisson, groupkfold, witch - 0.46152

V18: weights = 0.1, 0.1, 0.8 - tweedie, groupkfold, witch - 0.46140

V19: weights = 0.05, 0.1, 0.85 - tweedie, groupkfold, witch - 0.46160

V20: weights = 0.1, 0.1, 0.8 - tweedie, groupkfold, dark_witch - 0.45802

V21: weights = 0.05, 0.1, 0.85 - tweedie, groupkfold, dark_witch - 0.45815

V22: weights = 0.15, 0.1, 0.75 - tweedie, groupkfold, dark_witch - 0.45805

V23: weights = 0.1, 0.15, 0.75 - tweedie, groupkfold, dark_witch - 0.45816

V24: weights = 0.1, 0.05, 0.85 - tweedie, groupkfold, dark_witch - **0.45799

V25: weights = 0.1, 0.9 - tweedie, dark_witch - 0.45806

## Imports

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/m5-three-shades-of-dark-darker-magic/lgb_model_WI_2_v1.bin
/kaggle/input/m5-three-shades-of-dark-darker-magic/__results__.html
/kaggle/input/m5-three-shades-of-dark-darker-magic/test_CA_1.pkl
/kaggle/input/m5-three-shades-of-dark-darker-magic/test_WI_3.pkl
/kaggle/input/m5-three-shades-of-dark-darker-magic/test_CA_2.pkl
/kaggle/input/m5-three-shades-of-dark-darker-magic/test_TX_2.pkl
/kaggle/input/m5-three-shades-of-dark-darker-magic/__notebook__.ipynb
/kaggle/input/m5-three-shades-of-dark-darker-magic/test_TX_3.pkl
/kaggle/input/m5-three-shades-of-dark-darker-magic/__resultx__.html
/kaggle/input/m5-three-shades-of-dark-darker-magic/lgb_model_WI_1_v1.bin
/kaggle/input/m5-three-shades-of-dark-darker-magic/test_WI_1.pkl
/kaggle/input/m5-three-shades-of-dark-darker-magic/test_TX_1.pkl
/kaggle/input/m5-three-shades-of-dark-darker-magic/lgb_model_WI_3_v1.bin
/kaggle/input/m5-three-shades-of-dark-darker-magic/lgb_model_CA_3_v1.bin
/kaggle/input/m5-three-shades-of-dark-darker-ma

## Read CSV files

In [2]:
# dark witch 0.45863 - https://www.kaggle.com/marutama/m5-dark-witch-time-by-store
dark_witch = pd.read_csv('/kaggle/input/m5-dark-witch-time-by-store/submission.csv')
# groupkfold 0.47474 - https://www.kaggle.com/ramitjaal/m5-groupkfold-0-47474
groupkfold = pd.read_csv('/kaggle/input/m5-groupkfold-0-47474/submission.csv')
# tweedie 0.48721 - https://www.kaggle.com/kernelgenerator/m5-accuracy-tweedie-is-back
tweedie = pd.read_csv('/kaggle/input/m5-accuracy-tweedie-is-back/submission.csv')

## Old CSV files 
# witch 0.46193 - https://www.kaggle.com/kyakovlev/m5-witch-time/output
# witch = pd.read_csv('/kaggle/input/m5-witch-time/submission.csv')
# forecasting_acc 0.48587 - https://www.kaggle.com/jafarib/m5-forecasting-accuracy
# forecasting_acc = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/ss-001.csv')
# poisson 0.48874 - https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50/output
# poisson = pd.read_csv('/kaggle/input/m5-first-public-notebook-under-0-50/submission.csv')
# three_shades 0.47506 - https://www.kaggle.com/kyakovlev/m5-three-shades-of-dark-darker-magic
# three_shades = pd.read_csv('/kaggle/input/m5-three-shades-of-dark-darker-magic/submission_v1.csv')


In [3]:
print(tweedie.head())

                            id        F1        F2        F3        F4  \
0  FOODS_1_001_CA_1_validation  0.883426  0.888413  0.894976  0.885251   
1  FOODS_1_001_CA_2_validation  0.987750  1.111154  1.037725  1.352104   
2  FOODS_1_001_CA_3_validation  1.142896  1.072823  1.019548  0.959430   
3  FOODS_1_001_CA_4_validation  0.427192  0.359268  0.380275  0.385565   
4  FOODS_1_001_TX_1_validation  0.180103  0.177903  0.194126  0.194847   

         F5        F6        F7        F8        F9  ...       F19       F20  \
0  1.148076  1.166659  1.068241  0.997050  0.966213  ...  1.099531  1.293560   
1  1.333528  1.387823  1.397047  1.006586  1.005386  ...  1.287755  1.461417   
2  1.031894  1.195283  0.962712  1.045742  1.125412  ...  1.184673  1.541449   
3  0.438049  0.435912  0.452841  0.404452  0.424189  ...  0.472227  0.498934   
4  0.244718  0.253934  0.203959  0.502333  0.400345  ...  0.384761  0.377675   

        F21       F22       F23       F24       F25       F26       F27  \

In [4]:
print(groupkfold.head())

                              id        F1        F2        F3        F4  \
0  HOBBIES_1_001_CA_1_validation  0.823854  0.740466  0.737976  0.729739   
1  HOBBIES_1_002_CA_1_validation  0.204654  0.190715  0.181696  0.199629   
2  HOBBIES_1_003_CA_1_validation  0.419065  0.377333  0.385724  0.395076   
3  HOBBIES_1_004_CA_1_validation  1.609477  1.304986  1.339988  1.481931   
4  HOBBIES_1_005_CA_1_validation  0.942782  0.835392  0.874290  0.913130   

         F5        F6        F7        F8        F9  ...       F19       F20  \
0  0.935129  1.097512  1.193458  0.881441  0.939250  ...  0.847881  1.115740   
1  0.236983  0.272535  0.301582  0.219722  0.210785  ...  0.223283  0.282587   
2  0.536736  0.656096  0.680107  0.463931  0.441632  ...  0.567225  0.684923   
3  1.879556  2.730345  3.087827  1.641049  1.454909  ...  1.897165  2.569217   
4  1.081788  1.410473  1.569676  1.000622  1.051679  ...  1.067642  1.432376   

        F21       F22       F23       F24       F25       F26 

In [5]:
print(dark_witch.head())

                              id        F1        F2        F3        F4  \
0  HOBBIES_1_001_CA_1_validation  0.761751  0.706031  0.718703  0.742224   
1  HOBBIES_1_002_CA_1_validation  0.193665  0.186017  0.181720  0.177463   
2  HOBBIES_1_003_CA_1_validation  0.449849  0.419203  0.416102  0.409177   
3  HOBBIES_1_004_CA_1_validation  1.636782  1.269215  1.325452  1.492560   
4  HOBBIES_1_005_CA_1_validation  0.951531  0.848055  0.855237  0.921606   

         F5        F6        F7        F8        F9  ...       F19       F20  \
0  0.926323  1.017605  1.142859  0.847462  0.856534  ...  0.770908  0.957520   
1  0.221321  0.264605  0.294201  0.236565  0.222527  ...  0.207496  0.226067   
2  0.577780  0.742661  0.757365  0.491215  0.496076  ...  0.549582  0.710843   
3  1.908005  2.879368  3.170365  1.761563  1.480806  ...  1.861259  2.713125   
4  1.062124  1.470185  1.509062  0.974622  0.969054  ...  1.039325  1.621189   

        F21       F22       F23       F24       F25       F26 

# Attach weights
Calculate new forecasting by taking weighted sum of each model predictions

In [ ]:
tweedie = tweedie.sort_values(by = 'id').reset_index(drop = True)
groupkfold = groupkfold.sort_values(by = 'id').reset_index(drop = True)
dark_witch = dark_witch.sort_values(by = 'id').reset_index(drop = True)
sub = dark_witch.copy()

for i in sub.columns :
    if i != 'id' :
        sub[i] = 0.1 * tweedie[i] + 0.05 * groupkfold[i] + 0.85 * dark_witch[i]
        
sub.to_csv('submission_ensemble_v24.csv', index = False)
sub.head()